<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/HPTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from newsolver import predict_community_fullnp
import numpy as np
import pandas as pd
import random as rd
from numba import njit
from numba.typed import List
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pickle
import torch.optim as optim
import time
from math import sqrt
import matplotlib.pyplot as plt
import logging
import tqdm

logging.basicConfig(filename='output.log', level=logging.DEBUG)

data = pd.read_excel('RealData.xlsx', index_col=0)
specs = data.columns.tolist()
trimmed_specs = []

for i in range(len(specs)):
    if data.iloc[:,i].astype(bool).sum() >= 85:
        trimmed_specs.append(specs[i])
dim1 = len(trimmed_specs)

typed_trimmed_specs = List()
[typed_trimmed_specs.append(x) for x in trimmed_specs]

@njit()
def get_LT(full_ar):
    ar = []
    for i in range(len(full_ar)):
        for j in range(i):
            ar.append(full_ar[i][j])
    return ar

@njit()
def generate_matrix(comm, tolerance):
    dim = len(comm)
    ar = np.zeros((dim,dim))

    for i in range(dim):
        for j in range(i+1):
            if i == j:
                ar[i][j] = 0
            else:
                r = rd.random()
                # m = mult[i*dim1+j]
                ar[i][j] = r
                ar[j][i] = (1-r)

    return ar

def datagen():
    lm = generate_matrix(typed_trimmed_specs, 0)
    cm = predict_community_fullnp(lm, trimmed_specs, verb=False)
    return (cm, get_LT(lm))

# select CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if str(device) == 'cuda:0':
	print('CUDA device selected!')
elif str(device) == 'cpu':
	print('CUDA device not available. CPU selected')

# hyperparam = 470

class MyNet(nn.Module):
    def __init__(self, hyperparam):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, hyperparam)
        self.fc2 = nn.Linear(hyperparam, 231*461)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

mytest_x = []
mytest_y = []

for i in range(50):
      x, y = datagen()
      mytest_x.append(torch.from_numpy(x).float().to(device))
      mytest_y.append(torch.FloatTensor(y).to(device))

def test_net(model, test_x, test_y):
    test_loss = 0
    for i in range(len(test_x)):
      input, true_y = test_x[i], test_y[i]

      output = model(input).to(device)
      loss = criterion(output, true_y).to(device)
      test_loss += sqrt((loss.item())/(231*461))

    return test_loss/(len(test_x))

def testconfig(model):
  s_arr = []
  for i in range(3000):
      optimizer.zero_grad()
      x, y = datagen()
      input = torch.from_numpy(x).float().to(device)
      true_y = torch.FloatTensor(y).to(device)

      output = model(input).to(device)

      loss = criterion(output, true_y).to(device)
      s = sqrt((loss.item())/(231*461))
      s_arr.append(s)
      print(f'Epoch {i}: {s}')
      loss.backward()

      optimizer.step()
  acc = test_net(model, mytest_x, mytest_y)
  return acc

for it in range(470, 4630, 10):
  hyperparam = it
  net = MyNet(hyperparam).to(device)
  criterion = nn.MSELoss(reduction='sum')
  optimizer = optim.Adam(net.parameters(), lr=1e-4)

  acc = testconfig(net)
  logging.info(f'Hidden Layer n={it}')
  logging.info(f'Test Acc: {acc}')
  
